In [11]:
import json
import asyncio
import websockets

from tvDatafeed import TvDatafeed

tv = TvDatafeed()

In [2]:
def write_raw_data(raw_data, symbol):
    with open(f"{symbol}.json", 'w') as file:
        file.write(raw_data)

def raw_data_to_json(raw_data):
    results = raw_data.split("\n")
    sections = []
    for result in results:
        parsed = tv.parse_m_format(result)
        sections.append(parsed)
    
    return sections

def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        elif isinstance(item, dict):
            if "v" in item:
                flattened.append(item["v"])
            if "p" in item:
                if "v" in item["p"][1]:
                    flattened.append(item["p"][1]["v"])
        else:
            flattened.append(item)
    return flattened

def get_financial_data(
    symbol: str,
    exchange: str,
) -> dict:
    """get financial ratios

    Args:
        symbol (str): symbol name
        exchange (str, optional): exchange, not required if symbol is in format EXCHANGE:SYMBOL. Defaults to None.

    Returns:
        dict
    """
    symbol = tv.format_symbol(symbol=symbol, exchange=exchange)

    tv.create_connection()

    tv.send_set_auth_token()
    tv.send_quote_create_session_msg()
    tv.send_message(
        "quote_add_symbols",
        [tv.session, symbol],
    )
    raw_data = tv.receive_data(sentinel="quote_completed")

    return raw_data

def format_float(value, decimal_places=2, default="N/A"):
    """
    Formats a float value to the specified number of decimal places.
    Returns the default string if the value is None.
    
    :param value: The float value to format
    :param decimal_places: The number of decimal places to display
    :param default: The default string to return if value is None
    :return: A formatted string or the default string
    """
    if value is None:
        return default
    format_spec = f".{decimal_places}f"
    return f"{value:{format_spec}}"

financial_data_raw = get_financial_data("BATS", "LSE")

sections = raw_data_to_json(financial_data_raw)
write_raw_data(json.dumps(sections, indent=4), "sections")

flattened_sections = flatten_list(sections)

write_raw_data(json.dumps(flattened_sections, indent=4), "flattened_sections")

flattened_dict = {key: value for d in flattened_sections for key, value in d.items()}
write_raw_data(json.dumps(flattened_dict, indent=4), "flattened_dict")

In [3]:
def print_financials(financials_dict):

    share_price = financials_dict.get('lp', None)
    currency = financials_dict.get('currency_code', None)
    shares_outstanding = financials_dict.get('total_shares_outstanding_current', None)
    average_volume = financials_dict.get('average_volume', None)
    market_cap = financials_dict.get('market_cap_calc', None)
    country_code = financials_dict.get('country_code', None)
    isin = financials_dict.get('isin', None)
    exhange = financials_dict.get('exchange', None)
    figi = financials_dict.get('figi', None)
    figi_exchange = figi.get("exchange-level", None) if figi else None
    figi_composite = figi.get("country-composite", None) if figi else None
    beta_1yr = financials_dict.get('beta_1_year', None)
    description = financials_dict.get('description', None)

    print(f"Share Price:          {share_price} ({currency})")
    print(f"Shares Outstanding:   {shares_outstanding}")
    print(f"Average Volume:       {average_volume}")
    print(f"Market Cap:           {market_cap}")
    print(f"Country Code:         {country_code}")
    print(f"ISIN:                 {isin}")
    print(f"Exchange:             {exhange}")
    if figi:
        print(f"FIGI Exchnage:        {figi_exchange}")
        print(f"FIGI Composite:       {figi_composite}")
    print(f"Beta 1 Year:          {format_float(beta_1yr)}")
    print(f"Description:          {description}")

    # Valuation ratios
    price_earnings_ratio = financials_dict.get("price_earnings", None)
    price_sales_ratio = financials_dict.get("price_sales_current", None)
    price_cash_flow_ratio = financials_dict.get("price_cash_flow_current", None)
    price_book_ratio = financials_dict.get("price_book_current", None)
    ev = financials_dict.get("enterprise_value_current", None)
    ev_ebitda = financials_dict.get("enterprise_value_ebitda_current", None)
    currency = financials_dict.get("currency", None)

    print("Valuation Ratios")
    print(f"  P/E:        {format_float(price_earnings_ratio)}")
    print(f"  P/S:        {format_float(price_sales_ratio)}")
    print(f"  P/B:        {format_float(price_book_ratio)}")
    print(f"  P/CF:       {format_float(price_cash_flow_ratio)}")
    print(f"  EV:         {format_float(ev)} ({currency})")
    print(f"  EV/EBITDA:  {format_float(ev_ebitda)}")

    # Profitability ratios
    return_on_assets = financials_dict.get("return_on_assets_current", None)
    return_on_equity = financials_dict.get("return_on_equity_current", None)
    return_on_invested_capital = financials_dict.get("return_on_invested_capital_current", None)
    gross_margin = financials_dict.get("gross_margin_current", None)
    operating_margin = financials_dict.get("operating_margin_current", None)
    ebitda_margin = financials_dict.get("ebitda_margin_current", None)
    net_margin = financials_dict.get("net_margin_current", None)

    print("\nProfitability Ratios")
    print(f"  Return on Assets:               {format_float(return_on_assets)}%")
    print(f"  Return on Equity:               {format_float(return_on_equity)}%")
    print(f"  Return on Invested Capital:     {format_float(return_on_invested_capital)}%")
    print(f"  Gross Margin:                   {format_float(gross_margin)}%")
    print(f"  Operating Margin:               {format_float(operating_margin)}%")
    print(f"  EBITDA Margin:                  {format_float(ebitda_margin)}%")
    print(f"  Net Margin:                     {format_float(net_margin)}%")


    # Liquidity ratios
    quick_ratio = financials_dict.get("quick_ratio", None)
    current_ratio = financials_dict.get("current_ratio", None)
    inventory_turnover = financials_dict.get("invent_turnover_current", None)
    asset_turnover = financials_dict.get("asset_turnover_current", None)

    print("\nLiquidity Ratios")
    print(f"  Quick Ratio:         {format_float(quick_ratio)}")
    print(f"  Current Ratio:       {format_float(current_ratio)}")
    print(f"  Inventory Turnover:  {format_float(inventory_turnover)}")
    print(f"  Asset Turnover:      {format_float(asset_turnover)}")

    # Solvency ratios
    debt_assets_ratio = financials_dict.get("debt_to_asset_current", None)
    debt_equity_ratio = financials_dict.get("debt_to_equity_current", None)
    long_term_debt_assets_ratio = financials_dict.get("long_term_debt_to_assets_current", None)
    long_term_debt_equity_ratio = financials_dict.get("long_term_debt_to_equity_current", None)

    print("\nSolvency Ratios")
    print(f"  Debt to Assets Ratio:      {format_float(debt_assets_ratio)}")
    print(f"  Debt to Equity Ratio:      {format_float(debt_equity_ratio)}")
    print(f"  LT Debt to Assets Ratio:   {format_float(long_term_debt_assets_ratio)}")
    print(f"  LT Debt to Equity Ratio:   {format_float(long_term_debt_equity_ratio)}")

print_financials(flattened_dict)

Share Price:          3348 (GBX)
Shares Outstanding:   2200558021
Average Volume:       6503727.1
Market Cap:           7366658409999.998
Country Code:         GB
ISIN:                 GB0002875804
Exchange:             LSE
FIGI Exchnage:        BBG000BG9N74
FIGI Composite:       BBG000BG9MW8
Beta 1 Year:          0.58
Description:          BRITISH AMERICAN TOBACCO ORD GBP0.25
Valuation Ratios
  P/E:        N/A
  P/S:        2.84
  P/B:        1.37
  P/CF:       8.31
  EV:         107552584100.00 (GBP)
  EV/EBITDA:  8.46

Profitability Ratios
  Return on Assets:               -10.46%
  Return on Equity:               -21.96%
  Return on Invested Capital:     -14.13%
  Gross Margin:                   58.19%
  Operating Margin:               34.63%
  EBITDA Margin:                  47.76%
  Net Margin:                     36.23%

Liquidity Ratios
  Quick Ratio:         0.58
  Current Ratio:       0.88
  Inventory Turnover:  1.68
  Asset Turnover:      0.20

Solvency Ratios
  Debt to Asse

### Attempting to get multiple symbols in one connection. Needs work

In [22]:
async def send_messages(websocket, messages):
    print("sending messages")

    for message in messages:
        await websocket.send(message)
        print(f"Sent: {message}")



async def receive_messages(websocket):
    print("receiving messages")

    messages = []

    while True:
        try:
            message = await asyncio.wait_for(websocket.recv(), timeout=10)
            print(f"Received: {message}")
            messages.append(message)
            await asyncio.sleep(1)
        except asyncio.TimeoutError:
            print("No messages received in 10 seconds. Closing connection.")
            await websocket.close()
            break
        except websockets.exceptions.ConnectionClosed:
            print("Error: Connection closed.")
        except Exception as e:
            print(f"Error: {e}")

    # return messages

async def get_multi_financial_data(symbols: list[str]):

    # tv.create_connection()

    # tv.send_set_auth_token()
    # tv.send_quote_create_session_msg()

    sign_in_url = "https://www.tradingview.com/accounts/signin/"
    search_url = "https://symbol-search.tradingview.com/symbol_search/v3/?text={}&hl=1&exchange={}&lang=en&&search_type=undefined&domain=production&sort_by_country=US"
    ws_headers = json.dumps({"Origin": "https://data.tradingview.com"})
    signin_headers = {"Referer": "https://www.tradingview.com"}
    ws_timeout = 5

    ws = websockets.create_connection(
        "wss://data.tradingview.com/socket.io/websocket",
        headers=ws_headers,
        timeout=ws_timeout,
    )



    send_task = send_messages(tv.ws, symbols)
    receive_task = receive_messages(tv.ws)

    await asyncio.gather(send_task, receive_task)

    # print("Final received messages:", received_messages[1])

end_message = "!!!END!!!"


symbols = [
    tv.format_symbol(symbol="BATS", exchange="LSE"),
    tv.format_symbol(symbol="IMB", exchange="LSE"),
    tv.format_symbol(symbol="JEMA", exchange="LSE"),
    tv.format_symbol(symbol="NWG", exchange="LSE"),
]


await get_multi_financial_data(symbols)

# financial_data_raw = await get_multi_financial_data(symbols)
# write_raw_data(financial_data_raw, "multi_financial_data")

# # not working - need to rework for multiple symbols
# multi_sections = raw_data_to_json(financial_data_raw)
# write_raw_data(json.dumps(multi_sections, indent=4), "multi_sections")

sending messages
receiving messages


TypeError: object int can't be used in 'await' expression